### Data Wrangling: Clean, Transform, Merge, Reshape

Much of the programming work in data analysis and modeling is spent on data preparation loading, cleaning, transforming, and rearranging. Sometimes the way that data is stored in files or databases is not the way you need it for a data processing application.

pandas along with the Python standard library provide you with a high-level, flexible, and high-performance set of core manipulations and algorithms to enable you to wrangle data into the right form without much trouble.

In [4]:
import pandas as pd
import numpy as np

#Combining and Merging Data Sets

#1. pandas.merge :
    #connects rows in DataFrames based on one or more keys.


df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})

df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                 'data2': range(3)})

print("First Dataframe: \n",df1)

print("Second Dataframe: \n",df2)

#This is an example of a many-to-one merge situation; the data in df1 has multiple rows labeled a and b, 
#whereas df2 has only one row for each value in the key column. Calling merge with these objects we obtain:

print("Many-to-one Merge: \n",pd.merge(df1, df2))

First Dataframe: 
    data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   a
6      6   b
Second Dataframe: 
    data2 key
0      0   a
1      1   b
2      2   d
Many-to-one Merge: 
    data1 key  data2
0      0   b      1
1      1   b      1
2      6   b      1
3      2   a      0
4      4   a      0
5      5   a      0


In [6]:
##Note that I didn’t specify which column to join on. If not specified, merge uses the 
#overlapping column names as the keys. It’s a good practice to specify explicitly, though:

print("Many-to-one Merge: \n",pd.merge(df1, df2, on = 'key'))

Many-to-one Merge: 
    data1 key  data2
0      0   b      1
1      1   b      1
2      6   b      1
3      2   a      0
4      4   a      0
5      5   a      0


In [5]:
#Many-to-many merges

df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                 'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                          'data2': range(5)})

print("First Data Frame:\n",df1)
print("First Data Frame:\n",df2)


print("Left Outer Join: \n",pd.merge(df1, df2, on='key', 
                                     how='left'))
print("Inner Join: \n",pd.merge(df1, df2, on='key', how='inner'))

First Data Frame:
    data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   b
First Data Frame:
    data2 key
0      0   a
1      1   b
2      2   a
3      3   b
4      4   d
Left Outer Join: 
     data1 key  data2
0       0   b    1.0
1       0   b    3.0
2       1   b    1.0
3       1   b    3.0
4       2   a    0.0
5       2   a    2.0
6       3   c    NaN
7       4   a    0.0
8       4   a    2.0
9       5   b    1.0
10      5   b    3.0
Inner Join: 
    data1 key  data2
0      0   b      1
1      0   b      3
2      1   b      1
3      1   b      3
4      5   b      1
5      5   b      3
6      2   a      0
7      2   a      2
8      4   a      0
9      4   a      2


In [15]:
#To merge with multiple keys, pass a list of column names:

left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                  'key2': ['one', 'two', 'one'],
                  'lval': [1, 2, 3]})

right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one','one', 'two'],
                   'rval': [4, 5, 6, 7]})

pd.merge(left, right, on=['key1', 'key2'], how='outer')
                           

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [12]:
#To merge with multiple keys, when column names are not same:

left = pd.DataFrame({'key11': ['foo', 'foo', 'bar'],
                  'lval1': [1, 2, 3]})

right = pd.DataFrame({'key22': ['foo', 'foo', 'bar', 'bar'],
                   'rva2l': [4, 5, 6, 7]})

pd.merge(left,right, left_on='key11',right_on='key22', 
         how='outer')

,key11,lval1,key22,rva2l
0,foo,1,foo,4
1,foo,1,foo,5
2,foo,2,foo,4
3,foo,2,foo,5
4,bar,3,bar,6
5,bar,3,bar,7


In [8]:
#To merge with multiple keys, when column names are not same:

left = pd.DataFrame({'key11': ['foo', 'foo', 'bar'],
                  'key21': ['one', 'two', 'one'],
                  'lval1': [1, 2, 3]})

right = pd.DataFrame({'key21': ['foo', 'foo', 'bar', 'bar'],
                   'key22': ['one', 'one','one', 'two'],
                   'rva2l': [4, 5, 6, 7]})

pd.merge(left,right, left_on=['key11', 'key21'],
         right_on=['key21', 'key22']
         , how='outer')

,key11,key21_x,lval1,key21_y,key22,rva2l
0,foo,one,1.0,foo,one,4.0
1,foo,one,1.0,foo,one,5.0
2,foo,two,2.0,NaN,NaN,NaN
3,bar,one,3.0,bar,one,6.0
4,NaN,NaN,NaN,bar,two,7.0


In [3]:
#2. Concatenating Along an Axis

#Another kind of data combination operation is alternatively referred to as concatenation, binding, or stacking. 
#NumPy has a concatenate function for doing this with raw NumPy arrays:

#The concat function in pandas provides you hte same functionality

s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6,7], index=['f', 'g','a'])

print("First Series:\n",s1)
print("Second Series:\n",s2)
print("Third Series:\n",s3)

print("Concatinated Series: \n",pd.concat([s1, s2, s3]))

First Series:
 a    0
b    1
dtype: int64
Second Series:
 c    2
d    3
e    4
dtype: int64
Third Series:
 f    5
g    6
a    7
dtype: int64
Concatinated Series: 
 a    0
b    1
c    2
d    3
e    4
f    5
g    6
a    7
dtype: int64


In [4]:
#By default concat works along axis=0, producing another Series. If you pass axis=1, the
#result will instead be a DataFrame (axis=1 is the columns):


print("Concatinated Series on axis = 1: \n",
      pd.concat([s1, s2, s3], axis=1))

Concatinated Series on axis = 1: 
      0    1    2
a  0.0  NaN  7.0
b  1.0  NaN  NaN
c  NaN  2.0  NaN
d  NaN  3.0  NaN
e  NaN  4.0  NaN
f  NaN  NaN  5.0
g  NaN  NaN  6.0


In [4]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['a', 'd', 'e'])
s3 = pd.Series([5, 6], index=['a', 'g'])

print("First Series:\n",s1)
print("Second Series:\n",s2)
print("Third Series:\n",s3)

print("Concatinated Series on axis = 1: \n",
      pd.concat([s1, s2, s3], axis=1))

First Series:
 a    0
b    1
dtype: int64
Second Series:
 a    2
d    3
e    4
dtype: int64
Third Series:
 a    5
g    6
dtype: int64
Concatinated Series on axis = 1: 
      0    1    2
a  0.0  2.0  5.0
b  1.0  NaN  NaN
d  NaN  3.0  NaN
e  NaN  4.0  NaN
g  NaN  NaN  6.0


In [31]:
#Duplicates

data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})

print("Data Frame with duplicate values: \n",data)

Data Frame with duplicate values: 
     k1  k2
0  one   1
1  one   1
2  one   2
3  two   3
4  two   3
5  two   4
6  two   4


In [34]:
#Identifying the duplicate values:

#The DataFrame method duplicated returns a boolean Series indicating whether each
#row is a duplicate or not:

print("Identify duplicate values: \n",data.duplicated())

Identify duplicate values: 
 0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool


In [36]:
#drop_duplicates returns a DataFrame where the duplicated array is True:

print("Drop Duplicates: \n",data.drop_duplicates())

Drop Duplicates: 
     k1  k2
0  one   1
2  one   2
3  two   3
5  two   4


In [38]:
#Both of these methods by default consider all of the columns; alternatively you can specify 
#any subset of them to detect duplicates

print("Drop K1 Duplicate:\n ",data.drop_duplicates(['k1']))

Drop K1 Duplicate:
      k1  k2
0  one   1
3  two   3


In [42]:
#duplicated and drop_duplicates by default keep the first observed value combination.
#Passing take_last=True will return the last one:

data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2
1,one,1
2,one,2
4,two,3
6,two,4


In [46]:
#Replacing Values

#Filling in missing data with the fillna method can be thought of as a special case of more general value replacement.

data = pd.Series([1., -999., 2., -999., -1000., 3.])

print(data)

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64


In [49]:
#The -999 values might be sentinel values for missing data. To replace these with NA
#values that pandas understands, we can use replace, producing a new Series:

print("Replace -999 with NA :\n",data.replace(-999, np.nan))

print("Replace -999 and -1000 with NA :\n",data.replace([-999,-1000], 
                                                        np.nan))

Replace -999 with NA :
 0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64
Replace -999 and -1000 with NA :
 0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64


In [51]:
#To use a different replacement for each value, pass a list of substitutes:

print("Replace -999 and -1000 with NA :\n",data.replace([-999,-1000], 
                                                        [np.nan,0]))

Replace -999 and -1000 with NA :
 0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64


In [54]:
#Detecting and Filtering Outliers

np.random.seed(12345)

data = pd.DataFrame(np.random.randn(1000, 4))

print("Data Frame describe: \n",data.describe())

Data Frame describe: 
                  0            1            2            3
count  1000.000000  1000.000000  1000.000000  1000.000000
mean     -0.067684     0.067924     0.025598    -0.002298
std       0.998035     0.992106     1.006835     0.996794
min      -3.428254    -3.548824    -3.184377    -3.745356
25%      -0.774890    -0.591841    -0.641675    -0.644144
50%      -0.116401     0.101143     0.002073    -0.013611
75%       0.616366     0.780282     0.680391     0.654328
max       3.366626     2.653656     3.260383     3.927528


In [61]:
#Computing Indicator/Dummy Variables

#Another type of transformation for statistical modeling or machine learning applications is 
#converting a categorical variable into a “dummy” or “indicator” matrix. If a column in a DataFrame 
#has k distinct values, you would derive a matrix or DataFrame containing k columns containing 
#all 1’s and 0’s. pandas has a get_dummies function for doing this.

df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                'data1': range(6)})


print("Dummy Encoding: \n", pd.get_dummies(df['key']))

Dummy Encoding: 
    a  b  c
0  0  1  0
1  0  1  0
2  1  0  0
3  0  0  1
4  1  0  0
5  0  1  0


# GroupBy: Split, Apply, Combine

Simple aggregations can give you a flavor of your dataset, but often we would prefer to aggregate conditionally on some label or index: this is implemented in the so-called groupby operation. 

In [15]:
import pandas as pd
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [4]:
#he most basic split-apply-combine operation can be computed with the groupby() method of DataFrames, 
#passing the name of the desired key column:

df.groupby('key')

It returned a DataFrameGroupBy object. This object is where the magic is: you can think of it as a special view of the DataFrame, which is poised to dig into the groups but does no actual computation until the aggregation is applied. This "lazy evaluation" approach means that common aggregates can be implemented very efficiently in a way that is almost transparent to the user.

To produce a result, we can apply an aggregate to this DataFrameGroupBy object, which will perform the appropriate apply/combine steps to produce the desired result:

In [5]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


In [9]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [10]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

In [11]:
#Another useful pattern is to pass a dictionary mapping column names to operations to be applied on that column:

df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9
